# Classifier Tests Jupyter Notebook

In [20]:
import importlib

# Import myutils.py from mysklearn forlder
import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils

# Import mypytable.py from mysklearn folder
import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 

# Import myclassifiers.py from mysklearn folder
import mysklearn.myclassifiers
importlib.reload(mysklearn.myclassifiers)
from mysklearn.myclassifiers import MyRandomForestClassifier, MyDecisionTreeClassifier, MyNaiveBayesClassifier

# Import myevaluation.py from mysklearn folder
import mysklearn.myevaluation
importlib.reload(mysklearn.myevaluation)
import mysklearn.myevaluation as myevaluation

## Random Forest Classifier

In [23]:
# Get data from csv file
table = MyPyTable().load_from_file("input_files/winequality-red.csv")
y_col = table.get_column("quality", False)
x_cols = table.drop_col("quality")

# Split data into train and test cases
X_train, X_test, y_train, y_test = myutils.train_test_split(x_cols, y_col, .33)

# Use Random Forest to classify
testcase = MyRandomForestClassifier()
testcase.fit(X_train, y_train, X_test, y_test)
y_predicted = testcase.predict(X_test)
print(y_predicted)
print(y_test)



alue', 15.0, ['Leaf', 7.0, 4, 1071]], ['Value', 15.5, ['Leaf', 5.0, 2, 1071]], ['Value', 15.6, ['Leaf', 5.0, 2, 1071]], ['Value', 15.9, ['Leaf', 5.0, 1, 1071]]]
['Attribute', 'att0', ['Value', 4.6, ['Leaf', 4.0, 2, 1071]], ['Value', 4.7, ['Leaf', 6.0, 2, 1071]], ['Value', 4.9, ['Leaf', 7.0, 2, 1071]], ['Value', 5.0, ['Leaf', 4.0, 1, 1071]], ['Value', 5.1, ['Leaf', 6.0, 2, 1071]], ['Value', 5.2, ['Leaf', 6.0, 4, 1071]], ['Value', 5.3, ['Leaf', 7.0, 3, 1071]], ['Value', 5.4, ['Leaf', 7.0, 2, 1071]], ['Value', 5.6, ['Leaf', 5.0, 2, 1071]], ['Value', 5.8, ['Leaf', 5.0, 1, 1071]], ['Value', 5.9, ['Leaf', 6.0, 3, 1071]], ['Value', 6.0, ['Leaf', 5.0, 2, 1071]], ['Value', 6.1, ['Leaf', 6.0, 6, 1071]], ['Value', 6.2, ['Leaf', 6.0, 10, 1071]], ['Value', 6.3, ['Leaf', 5.0, 3, 1071]], ['Value', 6.4, ['Leaf', 6.0, 10, 1071]], ['Value', 6.5, ['Leaf', 6.0, 2, 1071]], ['Value', 6.6, ['Leaf', 5.0, 12, 1071]], ['Value', 6.7, ['Leaf', 5.0, 15, 1071]], ['Value', 6.8, ['Leaf', 6.0, 17, 1071]], ['Value', 6.

## Naive Bayes Classifier

In [3]:
importlib.reload(mysklearn.mypytable)
importlib.reload(mysklearn.myevaluation)
import numpy as np

# Get data from csv file
table = MyPyTable().load_from_file(os.path.join("input_files","winequality-red.csv"))
y_col = table.get_column("quality", False)
x_cols = table.drop_col("quality")

# Use Naive Bayes to classify
testcase = MyNaiveBayesClassifier()

#Returns x INDEXES
X_train, X_test = myevaluation.stratified_kfold_cross_validation(x_cols,y_col,n_splits=10)
X_train, X_test, y_train, y_test = myutils.getInstances(X_train, X_test, x_cols,y_col)

predicted_values = []
for i,fold in enumerate(X_train):
    train,test = myutils.normalize_values(X_train[i],X_test[i])
    testcase.fit(train,y_train[i])
    predicted_values.append(testcase.predict(test))

numCorrectPredictions = 0
numWrongPredictions = 0
for i,fold in enumerate(X_test):
    for index in range(len(fold)):
        values = [predicted_values[i][index], y_test[i][index]] #predicted/actual
        if(values[0]==values[1]):
            numCorrectPredictions = numCorrectPredictions+1
        else:
            numWrongPredictions = numWrongPredictions+1

accuracy = np.round((numCorrectPredictions)/(numCorrectPredictions+numWrongPredictions),3)
error_rate = np.round((numWrongPredictions)/(numCorrectPredictions+numWrongPredictions),3)

print("-----------------------------------------------------------")
print("Accuracy and Error Rate")
print("-----------------------------------------------------------")
print()
print("Naive Bayes: accuracy = {}, error rate = {}".format(accuracy,error_rate))

-----------------------------------------------------------
Accuracy and Error Rate
-----------------------------------------------------------

Naive Bayes: accuracy = 0.6, error rate = 0.4


## Decision Tree Classifier

In [4]:
importlib.reload(mysklearn.mypytable)
importlib.reload(mysklearn.myevaluation)
import numpy as np

# Get data from csv file
table = MyPyTable().load_from_file(os.path.join("input_files","winequality-red.csv"))
y_col = table.get_column("quality", False)
x_cols = table.drop_col("quality")

# TODO: NEED TO CHANGE X-COLS TO CATEGORICAL ATTRIBUTES

# Use Decision Tree to classify
testcase = MyDecisionTreeClassifier()

#Returns x INDEXES
X_train, X_test = myevaluation.stratified_kfold_cross_validation(x_cols,y_col,n_splits=10)
X_train, X_test, y_train, y_test = myutils.getInstances(X_train, X_test, x_cols,y_col)

predicted_values = []
for i,fold in enumerate(X_train):
    train = myutils.categorize_dataset(X_train[i])
    test = myutils.categorize_dataset(X_test[i])
    testcase.fit(train,y_train[i])
    predicted_values.append(testcase.predict(test))

#testcase.print_decision_rules(table.column_names[:-1],"quality")

numCorrectPredictions = 0
numWrongPredictions = 0
for i,fold in enumerate(X_test):
    for index in range(len(fold)):
        values = [predicted_values[i][index], y_test[i][index]] #predicted/actual
        if(values[0]==values[1]):
            numCorrectPredictions = numCorrectPredictions+1
        else:
            numWrongPredictions = numWrongPredictions+1

accuracy = np.round((numCorrectPredictions)/(numCorrectPredictions+numWrongPredictions),3)
error_rate = np.round((numWrongPredictions)/(numCorrectPredictions+numWrongPredictions),3)

print("Decision Tree: accuracy = {}, error rate = {}".format(accuracy,error_rate))

IF alcohol == 1 AND total sulfur dioxide == 1 THEN quality == 6.0
IF alcohol == 1 AND total sulfur dioxide == 2 THEN quality == 6.0
IF alcohol == 1 AND total sulfur dioxide == 3 THEN quality == 6.0
IF alcohol == 1 AND total sulfur dioxide == 4 THEN quality == 5.0
IF alcohol == 1 AND total sulfur dioxide == 5 THEN quality == 5.0
IF alcohol == 1 AND total sulfur dioxide == 6 THEN quality == 5.0
IF alcohol == 2 AND total sulfur dioxide == 1 AND sulphates == 1 THEN quality == 5.0
IF alcohol == 2 AND total sulfur dioxide == 1 AND sulphates == 2 THEN quality == 5.0
IF alcohol == 2 AND total sulfur dioxide == 1 AND sulphates == 3 THEN quality == 6.0
IF alcohol == 2 AND total sulfur dioxide == 1 AND sulphates == 4 THEN quality == 6.0
IF alcohol == 2 AND total sulfur dioxide == 1 AND sulphates == 5 THEN quality == 5.0
IF alcohol == 2 AND total sulfur dioxide == 2 THEN quality == 5.0
IF alcohol == 2 AND total sulfur dioxide == 3 THEN quality == 5.0
IF alcohol == 2 AND total sulfur dioxide == 4 A